In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.markers as markers
from datetime import timedelta, datetime
import os
import sqlalchemy
from sqlalchemy import create_engine
from dotenv import load_dotenv
#pd.set_option('display.max_rows', 1000)

In [ ]:
dotenv_local_path = './.env'
load_dotenv(dotenv_path=dotenv_local_path, verbose=True) 

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
# Connect to SQL

engine = create_engine("postgres://" + os.environ.get("AWS_POSTGRES_USER") + ":" + os.environ.get("AWS_POSTGRES_PASSWORD") + "@" + "amazon.coqoinqxklrf.us-east-1.rds.amazonaws.com:5432/postgres")

In [ ]:
# Create df from SQL

df = pd.read_sql("SELECT * FROM amazon.purchases_aws", con=engine, parse_dates = ['OrderDate', 'ShipDate'])
df.head()

In [ ]:
# Cumulative total spent from 2008-2019, inclusive

TotalSpent = df['ItemTotal'].sum()
TotalSpent

In [ ]:
# Total number of transactions

NumTrans = len(df['ItemTotal'])
NumTrans

In [ ]:
# Average spent

AvgSpent = TotalSpent/NumTrans
AvgSpent

In [ ]:
# Use groupby to sum purchases by date

df_DateTotal = df[['OrderDate', 'ItemTotal']]
df_DateTotal = df_DateTotal.groupby('OrderDate')
df_DateTotal = df_DateTotal.sum('ItemTotal')
df_DateTotal

In [ ]:
# Plot OrderDate & sum of purchases by date

fig, ax = plt.subplots(figsize=(15,5))
ax.plot(df_DateTotal.index, df_DateTotal.values)
ax.set_xlabel('Order Date')
ax.set_ylabel('Item Total')
ax.set_title('Daily Purchase Amount')
plt.show()
#ax.figure.savefig('./img/DailyPurchaseAmount.png', bbox_inches='tight', dpi=150)

In [ ]:
# Use groupby to sum taxes by date

df_DateTax = df[['OrderDate', 'Tax']]
df_DateTax = df_DateTax.groupby('OrderDate')
df_DateTax = df_DateTax.sum('Tax')
df_DateTax

In [ ]:
# Plot OrderDate and ItemTotal and OrderDate and PurchasePricePerUnit

fig, ax = plt.subplots(1,2, sharey = True)

ax[0].plot(df_DateTotal.index, df_DateTotal.values)
ax[1].plot(df_DateTax.index, df_DateTax.values, color = 'r')

ax[0].set_xlabel('Order Date')
ax[0].set_ylabel('$ Total')
ax[1].set_xlabel('Order Date')

ax[0].set_title('Daily Purchase Amount')
ax[1].set_title('Daily Tax Amount')

plt.show()

In [ ]:
# The previous graphs contained too much data since they were being plotted for each day. 
# Grouping will allow the information to be more legible.

# Purchases by Year 

df_OrderYear = df[['OrderDate', 'ItemTotal', 'OrderYear']]
df_OrderYear = df_OrderYear.sort_values('OrderDate')
df_OrderYear = df_OrderYear.groupby('OrderYear')
df_OrderYear = df_OrderYear['ItemTotal'].sum()
print(df_OrderYear)

In [ ]:
fig,ax = plt.subplots(figsize=(10,5))
ax.plot(df_OrderYear.index, df_OrderYear.values, marker = 'o')
ax.set_xlabel('Year')
ax.set_ylabel('$ Amount')
ax.set_title('Purchases by Year')
plt.xticks([x for x in df_OrderYear.index], rotation=0)

ax.yaxis.set_major_formatter(
    ticker.FuncFormatter(lambda y,p: format(int(y),','))
)

plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=(10,5))
ax.bar(df_OrderYear.index, df_OrderYear.values, color='darkorange')
ax.set_xlabel('Year')
ax.set_ylabel('$ Amount')
ax.set_title('Purchases by Year')
plt.xticks([x for x in df_OrderYear.index], rotation=0)

ax.yaxis.set_major_formatter(
    ticker.FuncFormatter(lambda y,p: format(int(y),','))
)

for i,j in zip(df_OrderYear.index,df_OrderYear.values):
    ax.annotate(format(int(j), ','), xy=(i,j), va='bottom', ha='center')

plt.show()
#ax.figure.savefig('./img/PurchasesByYear.png', bbox_inches='tight', dpi=150)

In [ ]:
# Purchases by Month

df_OrderMonth = df[['OrderDate', 'ItemTotal', 'OrderMonth']]
df_OrderMonth = df_OrderMonth.sort_values('OrderDate')
df_OrderMonth = df_OrderMonth.groupby('OrderMonth', as_index=False)
df_OrderMonth = df_OrderMonth['ItemTotal'].sum()
months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
df_OrderMonth['month_name'] = months
df_OrderMonth

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
ax.plot(df_OrderMonth.month_name, df_OrderMonth.ItemTotal, marker = 'o')
ax.set_xlabel('Month')
ax.set_ylabel('$ Amount')
ax.set_title('Purchases by Month')
plt.xticks([x for x in df_OrderMonth.index])

ax.yaxis.set_major_formatter(
    ticker.FuncFormatter(lambda y,p: format(int(y),','))
)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
ax.bar(df_OrderMonth.month_name, df_OrderMonth.ItemTotal, color='tomato')
ax.set_xlabel('Month')
ax.set_ylabel('$ Amount')
ax.set_title('Purchases by Month')
plt.xticks([x for x in df_OrderMonth.index])

ax.yaxis.set_major_formatter(
    ticker.FuncFormatter(lambda y,p: format(int(y),','))
)

for i,j in zip(df_OrderMonth.month_name, df_OrderMonth.ItemTotal):
    ax.annotate(format(int(j), ','), xy=(i,j), va='bottom', ha='center')

plt.show()
#ax.figure.savefig('./img/PurchasesByMonth.png', bbox_inches='tight', dpi=150)

In [ ]:
# Transactions by Year

df_TransYear = df[['OrderDate', 'ItemTotal', 'OrderYear']]
df_TransYear = df_TransYear.groupby('OrderYear', as_index=False)
df_TransYear = df_TransYear['ItemTotal'].size()
df_TransYear = df_TransYear.rename(columns={'size':'Trans'})
df_TransYear

In [ ]:
fig,ax = plt.subplots(figsize=(10,5))
ax.plot(df_TransYear.OrderYear, df_TransYear.Trans, marker = 'o')
ax.set_xlabel('Year')
ax.set_ylabel('Transactions')
ax.set_title('Transactions by Year')
plt.xticks([x for x in df_TransYear.OrderYear])

plt.show()
#ax.figure.savefig('./img/TransactionsByYear.png', bbox_inches='tight', dpi=150)

In [ ]:
# Transactions by Seller

df_TransSeller = df[['Seller']]
df_TransSeller = df_TransSeller.groupby(df_TransSeller['Seller'] == 'Amazon')
df_TransSeller = df_TransSeller.size()
df_TransSeller

In [ ]:
fig,ax = plt.subplots(figsize=(10,5))
ax.pie(df_TransSeller.values, labels = ['Third Party', 'Amazon'], autopct = '%1.1f%%')
ax.set_title('Transactions by Seller')
ax.axis('equal')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
ax.pie(df_TransSeller.values, autopct = '%1.1f%%')
ax.legend(
    #df_TransSeller.index,
    title = 'Seller',
    labels = ['Third Party', 'Amazon'],
    loc = 'center left',
    bbox_to_anchor = (.75,.5,0,0)
)
ax.set_title('Transactions by Seller')
ax.axis('equal')
plt.show()
#ax.figure.savefig('./img/TransactionsBySeller.png', bbox_inches='tight', dpi=150)

In [ ]:
# Purchases by Seller

df_PurchasesSeller = df[['Seller', 'ItemTotal']]
df_PurchasesSeller = df_PurchasesSeller.groupby(df_PurchasesSeller['Seller'] == 'Amazon')
df_PurchasesSeller = df_PurchasesSeller['ItemTotal'].sum()
df_PurchasesSeller

In [ ]:
fig,ax = plt.subplots(figsize=(10,5))
ax.pie(df_PurchasesSeller.values, autopct = '%1.1f%%')
ax.legend(
    #df_PurchasesSeller.index,
    title = 'Seller',
    labels = ['Third Party', 'Amazon'],
    loc = 'center left',
    bbox_to_anchor = (.75,.5,0,0)    
)
ax.set_title('Purchases by Seller')
ax.axis('equal')
plt.show()
#ax.figure.savefig('./img/PurchasesBySeller.png', bbox_inches='tight', dpi=150)

In [ ]:
# Purchases by Category
# Note: it is necessary to sort by reverse in the df so the plot can be arranged alphabetically
# from top to bottom

df_CategoryTotal = df[['Category', 'ItemTotal']]
df_CategoryTotal = df_CategoryTotal.groupby('Category')
df_CategoryTotal = df_CategoryTotal['ItemTotal'].sum()
df_CategoryTotal = df_CategoryTotal.sort_index(ascending = False)
df_CategoryTotal

In [ ]:
# Plot purchases by category

fig,ax = plt.subplots(figsize=(15,5))
ax.barh(df_CategoryTotal.index, df_CategoryTotal.values)
ax.set_xlabel('$ Amount')
ax.set_ylabel('Category')
ax.set_title('Purchases by Category')

ax.xaxis.set_major_formatter(
    ticker.FuncFormatter(lambda x,p: format(int(x),','))
)

for i,j in zip(df_CategoryTotal.values, df_CategoryTotal.index):
    ax.annotate(format(int(i), ','), xy=(i,j), va='center', ha='left')

plt.show()
#ax.figure.savefig('./img/PurchasesByCategory.png', bbox_inches='tight', dpi=150)

In [ ]:
# Purchases by category share

df_CategoryShare = df[['Category', 'ItemTotal']]
df_CategoryShare = df_CategoryShare.groupby('Category', as_index = False)
df_CategoryShare = df_CategoryShare['ItemTotal'].sum()
df_CategoryShare['Share'] = df_CategoryShare['ItemTotal'].div(df_CategoryShare['ItemTotal'].sum())*100
df_CategoryShare = df_CategoryShare.sort_values('Category', ascending = False)
df_CategoryShare

In [ ]:
# Plot purchases by category share

fig,ax = plt.subplots(figsize=(10,5))
ax.barh(df_CategoryShare['Category'],df_CategoryShare['Share'], color='tomato')
ax.set_xlabel('Percent')
ax.set_ylabel('Category')
ax.set_title('Purchases by Category Share')

for i,j in zip(df_CategoryShare['Share'], df_CategoryShare['Category']):
    ax.annotate(format(str(int(i))+'%', ''), xy=(i,j), va='center', ha='left')
    
ax.xaxis.set_major_formatter(
    ticker.FuncFormatter(lambda x,p: format(str(int(x))+'%',''))
)

plt.show()
#ax.figure.savefig('./img/PurchasesByCategoryShare.png', bbox_inches='tight', dpi=150)

In [ ]:
# Purchases by day of week

df_DayPur = df[['OrderDayIndex','OrderDayName','ItemTotal']]
df_DayPur = df_DayPur.groupby(['OrderDayName','OrderDayIndex'], as_index = False)
df_DayPur = df_DayPur['ItemTotal'].sum()
df_DayPur = df_DayPur.sort_values('OrderDayIndex')
df_DayPur

In [ ]:
# Plot purchases by day of week

fig,ax = plt.subplots(figsize=(10,5))
ax.bar(df_DayPur['OrderDayName'],df_DayPur['ItemTotal'], color='skyblue')
ax.set_xlabel('Day')
ax.set_ylabel('$ Amount')
ax.set_title('Purchases by Day')
plt.xticks(rotation = 0)

ax.yaxis.set_major_formatter(
    ticker.FuncFormatter(lambda y,p: format(int(y),','))
)

for i,j in zip(df_DayPur['OrderDayName'],df_DayPur['ItemTotal']):
    ax.annotate(format(int(j), ','), xy=(i,j), va='bottom', ha='center')

plt.show()
#ax.figure.savefig('./img/PurchasesByDay.png', bbox_inches='tight', dpi=150)

In [ ]:
# Avg transaction value by year

df_YearAvgTrans = df[['OrderYear','ItemTotal']]
df_YearAvgTrans = df_YearAvgTrans.groupby(['OrderYear'], as_index = False)
df_YearAvgTrans = df_YearAvgTrans['ItemTotal'].sum()
df_YearAvgTrans

In [ ]:
df_NumTransYear = df[['OrderYear', 'OrderID']].groupby('OrderYear').size()
df_NumTransYear

In [ ]:
df_YearAvgTrans['Trans'] = df_NumTransYear.values
df_YearAvgTrans

In [ ]:
df_YearAvgTrans['AvgPerTrans'] = df_YearAvgTrans['ItemTotal'].div(df_YearAvgTrans['Trans'])
df_YearAvgTrans

In [ ]:
# All of the above can be done more quickly using the following:

#df_YearAvgTrans = df[['OrderYear','ItemTotal']]
#df_YearAvgTrans = df_YearAvgTrans.groupby(['OrderYear'], as_index = False)['ItemTotal'].sum()

#df_YearOrders = df[['OrderYear','ItemTotal']].groupby('OrderYear')['ItemTotal'].size()
#df_YearAvgTrans['Trans'] = df_YearOrders.values

#df_YearAvgTrans['AvgTrans'] = df_YearAvgTrans['ItemTotal'].div(df_YearAvgTrans['Trans'])
#df_YearAvgTrans

In [ ]:
# Plot average per transaction by year

fig, ax = plt.subplots(figsize=(10,5))
ax.bar(df_YearAvgTrans['OrderYear'],df_YearAvgTrans['AvgPerTrans'], color='mediumaquamarine')
ax.set_xlabel('Year')
ax.set_ylabel('$ Amount')
ax.set_title('Average per Transaction per Year')
plt.xticks([x for x in df_YearAvgTrans['OrderYear']], rotation = 0)

ax.yaxis.set_major_formatter(
    ticker.FuncFormatter(lambda y,p: format(int(y),','))
)

for i,j in zip(df_YearAvgTrans['OrderYear'],df_YearAvgTrans['AvgPerTrans']):
    ax.annotate(format(int(j), ','), xy=(i,j), va='bottom', ha='center')

plt.show()
#ax.figure.savefig('./img/AveragePerTransPerYear.png', bbox_inches='tight', dpi=150)

In [ ]:
# Shipping method

df_Carriers = df['Carrier']
CarrierVC = df_Carriers.value_counts()
CarrierVC = CarrierVC.sort_index(ascending = False)
CarrierVC

In [ ]:
# Plot of shipping method

fig,ax = plt.subplots(figsize=(10,5))
ax.barh(CarrierVC.index, CarrierVC.values, color='steelblue')
ax.set_xlabel('Number of Orders')
ax.set_ylabel('Carrier')
ax.set_title('Shipping Method')

for i,j in zip(CarrierVC.values, CarrierVC.index):
    ax.annotate(format(int(i), ','), xy=(i,j), va='center', ha='left')
    
ax.xaxis.set_major_formatter(
    ticker.FuncFormatter(lambda x,p: format(int(x),','))
)

plt.show()
#ax.figure.savefig('./img/ShippingMethod.png', bbox_inches='tight', dpi=150)

In [ ]:
# Shipping method and avg transaction cost

df_CarrierTotal = df[['Carrier', 'ItemTotal']]
df_CarrierTotal = df_CarrierTotal.groupby('Carrier', as_index = False)['ItemTotal'].sum()

df_CarrierTrans = df[['Carrier', 'ItemTotal']].groupby('Carrier')['ItemTotal'].size()
df_CarrierTotal['NumTrans'] = df_CarrierTrans.values

df_CarrierTotal['AvgPerTrans'] = df_CarrierTotal['ItemTotal'].div(df_CarrierTotal['NumTrans']).round(2)

df_CarrierTotal = df_CarrierTotal.sort_values('Carrier', ascending = False)
df_CarrierTotal

In [ ]:
# Plot shipping method and avg transaction cost

fig,ax = plt.subplots(figsize=(10,5))
ax.barh(df_CarrierTotal['Carrier'], df_CarrierTotal['AvgPerTrans'], color='tomato')
ax.set_xlabel('$ Avg Per Transaction')
ax.set_ylabel('Carrier')
ax.set_title('Shipping Method & Average Price Per Transaction')

for i,j in zip(df_CarrierTotal['AvgPerTrans'], df_CarrierTotal['Carrier']):
    ax.annotate(format(int(i), ','), xy=(i,j), va='center', ha='left')
    
ax.xaxis.set_major_formatter(
    ticker.FuncFormatter(lambda x,p: format(int(x),','))
)

plt.show()
#ax.figure.savefig('./img/ShippingMethodPrice.png', bbox_inches='tight', dpi=150)